In [20]:
import numpy as np
import os
import cv2
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import os
from glob import glob
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.applications import ResNet50, DenseNet121, InceptionV3, VGG16, MobileNet, Xception
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

## Create Paths

In [8]:
#Cloud Path
path_to_train = '/home/ubuntu/project/chest_xray/train/'
path_to_test = '/home/ubuntu/project/chest_xray/test/'
path_to_val = '/home/ubuntu/project/chest_xray/val/'

In [21]:
#Mac Path
path_to_train = '/Users/ferhat/Desktop/X_RAY/data/train'
path_to_test = '/Users/ferhat/Desktop/X_RAY/data/test/'
path_to_val = '/Users/ferhat/Desktop/X_RAY/data/val/'

## Image Preprocessing

In [22]:
batch_size = 32
image_size = 64


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale=1./255)

valid_datagen = ImageDataGenerator(
        rescale=1./255) 

    
train_generator = train_datagen.flow_from_directory(
        path_to_train,
        target_size=(image_size,image_size),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        path_to_test,
        target_size=(image_size,image_size),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical')


validation_generator = valid_datagen.flow_from_directory(
        path_to_val,
        target_size=(image_size,image_size),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical')

print("Shape of training images {}".format(train_generator.image_shape))
print("Shape of test images {}".format(test_generator.image_shape))
print("Shape of validation images {}".format(validation_generator.image_shape))

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Shape of training images (64, 64, 1)
Shape of test images (64, 64, 1)
Shape of validation images (64, 64, 1)


## Create model

In [23]:
def create_model():
    global model
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same",
                 input_shape=image_size)) 
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.25))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(1024,activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(rate=0.4))
    model.add(Dense(2, activation="softmax"))
    

In [24]:
create_model()

TypeError: 'int' object is not iterable

##  Callbacks

In [6]:
NAME = 'X_ray_classifier-{}'.format('Adam CNN')

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

es = EarlyStopping(patience=25)

checkpoint = ModelCheckpoint('best_model.h5',save_best_only=True)

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max') 

## Train Model

In [8]:
epoch = 1

model.compile(optimizer = 'sgd', loss = 'mse', metrics = ['accuracy'])
history = model.fit_generator(
          train_generator,
          epochs=epoch,
          steps_per_epoch=train_generator.samples // batch_size,
          validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size,
          callbacks=[es, tensorboard, checkpoint])

Instructions for updating:
Use tf.cast instead.


ValueError: Error when checking input: expected conv2d_input to have shape (10, 10, 1) but got array with shape (64, 64, 1)

In [50]:
def create_plots(history):
    
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

NameError: name 'history' is not defined

## GridSearch

In [16]:
epochs = 1
batch_size = 32

In [24]:
clf = KerasClassifier(build_fn=create_model, epochs=epochs, verbose=0)

In [31]:
param_grid = {
    'clf__optimizer':['adam', 'sgd','rmsprop'],
    'dropout':[0.1, 0.2, 0.5],
    'clf__loss':['mse', 
                     'categorical_crossentropy']
}

In [32]:
(X_train, Y_train) = train_generator.next()

grid = GridSearchCV(clf, cv=2, param_grid=param_grid)
grid.fit(X_train, Y_train)

ValueError: clf__kernel_initializer is not a legal parameter

## Deploying to Production